# **Mapping of frames in WordNet synsets**

Frames estratti con la funzione getFrameSetForStudent() per il cognome Grandi:
| ID | Frame |
| --- | --- |
| 2658 | Suicide_attack |
| 1633 | Board_vehicle |
| 1260 | Simple_name |
| 1871 | Access_scenario |
| 652 | Eclipse |

Dato che: 
 - Suicide_attack
 - Board_vehicle 
 - Simple_name 
 - Access_scenario 

non sono presenti in WordNet, sono stati scelti rispettivamente:
- Suicide
- Vehicle
- Name
- Access

In pratica dovrò: associare dei synset al frame name, frame elements e LUs presi dai 5 del frameset.
Considerando come contesto di disambiguazione la frame name definition, frame element definition e LU definition, rispettivamente.
Come contesto dei sensi considero le varie definizioni del termine principale del frame name, la gloss, esempi e iponimi e iperonimi. E lo stesso per FE e LU


    

In [37]:
import nltk
from nltk.corpus import framenet as fn
from nltk.corpus import wordnet as wn



### Funzioni di utility per il preprocessing

In [59]:
# Remove stopwords and punctuation from a given sentence
def remove_stopwords(sentence):
    stop_words = []
    with open('/Users/jak/Documents/Uni/TLN/TLN/Radicioni/data/stop_words_FULL.txt', 'r') as f:
        for line in f:
            stop_words.append(line.strip())
    sentence = [w for w in sentence if w not in stop_words]
    return sentence

# Tokenize a given list of words
def tokenize(list):
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(list)):
        list[i] = tokenizer.tokenize(list[i])
    return list

# ---- POSSIBLY NOT NEEDED ----
# Get content words from a given sentence
def get_content_words(sentence):
    sentence = ''.join(sentence)
    sentence = set(sentence.lower().split())
    sentence = remove_stopwords(sentence)
    return sentence


## Estrazione del contesto dei sensi

Creo una lista di content words per un dato synset, cercando all'interno di:
- definizione del synset
- esempi del synset
- definizioni di iponimi e iperonimi
- esempi di iponimi e iperonimi

In [58]:
# Get context for a given synset
def get_context(synset):
    context = set(synset.name().split('.')[0].split('_'))
    context.update(synset.definition().lower().split())
    for example in synset.examples():
        context.update(example.lower().split())
    for hypo in synset.hyponyms():
        context.update(hypo.name().split('.')[0].split('_'))
        context.update(hypo.definition().lower().split())
        for example in hypo.examples():
            context.update(example.lower().split())
    for hyper in synset.hypernyms():
        context.update(hyper.name().split('.')[0].split('_'))
        context.update(hyper.definition().lower().split())
        for example in hyper.examples():
            context.update(example.lower().split())
    context = remove_stopwords(context)
    # context = tokenize(context) Maybe not needed
    for word in context:
        word = word.rstrip()
    return context

context = get_context(wn.synset('car.n.01'))
print(context)
print(len(context))
print(context[-10].strip())


['job', 'box-shaped', 'guzzler', 'traveling', 'automobile', 'equipped', 'engine', 'lap', 'seating', 'fallen', 'rails', 'low', 'people', 'powered', 'owned;', 'fast', 'luggage', 'fuel', '1908', 'parade', 'sports', 'small', 'subcompact', 'terrain', 'repair', 'steam-powered', 'rear', 'gasoline', 'cruise', 'bus', 'policemen', 'ford', 'loaner', 'pulls', 'used-car', 'seat', 'combustion', 'gas', 'efficiency', 'term', 'henry', 'minivan', 'sales', 'engine;', 'ambulance', 'stock', 'long', 'driven', 'replacement', 'lent', 'acceleration', 'compact', 'wheels;', 'radiotelephonic', 'races', 'wheeled', 'course', 'called', 'limousine', 'pace', 'high-performance', 'front', 'stanley', 'steamer', 'electricity', 'jeep', 'early', 'resembles', 'beach', 'top', 'seats;', 'built', 'high-powered', 'passengers', 'carriages', 'streets;', 'coupe', 'fixed', 'leads', 'chassis', 'regular', 'self-propelled', 'internal', 'cruiser', 'propelled', 'communications', 'hot', 'wagon', 'sedan', 'door', 'hardtop', 'competes', "de

Estraggo il contesto dei sensi per ogni synset di una parola data

In [60]:
# TODO: da cambiare di sihuro deh
# Get context for all synsets of a given word
def get_context_all(word):
    context = []
    for synset in wn.synsets(word):
        context.append(get_context(synset))
    return context

context = get_context_all('car')
print(context)


[['job', 'box-shaped', 'guzzler', 'traveling', 'automobile', 'equipped', 'engine', 'lap', 'seating', 'fallen', 'rails', 'low', 'people', 'powered', 'owned;', 'fast', 'luggage', 'fuel', '1908', 'parade', 'sports', 'small', 'subcompact', 'terrain', 'repair', 'steam-powered', 'rear', 'gasoline', 'cruise', 'bus', 'policemen', 'ford', 'loaner', 'pulls', 'used-car', 'seat', 'combustion', 'gas', 'efficiency', 'term', 'henry', 'minivan', 'sales', 'engine;', 'ambulance', 'stock', 'long', 'driven', 'replacement', 'lent', 'acceleration', 'compact', 'wheels;', 'radiotelephonic', 'races', 'wheeled', 'course', 'called', 'limousine', 'pace', 'high-performance', 'front', 'stanley', 'steamer', 'electricity', 'jeep', 'early', 'resembles', 'beach', 'top', 'seats;', 'built', 'high-powered', 'passengers', 'carriages', 'streets;', 'coupe', 'fixed', 'leads', 'chassis', 'regular', 'self-propelled', 'internal', 'cruiser', 'propelled', 'communications', 'hot', 'wagon', 'sedan', 'door', 'hardtop', 'competes', "d

In [92]:
# Get frame context for a given frame
def get_frame_context(frame):
    context = set(frame.definition.lower().split())
    context.update(frame.name.lower().split())
    # FEs = frame.FE.keys()
    # for fe in FEs:
    #     fed = frame.FE[fe]
    #     context.update(fed.definition.lower().split())
    #     context.update(fe.lower().split())
    for fe in frame.FE:
        context.update(frame.FE[fe].definition.lower().split())
        context.update(fe.lower().split())
    for lu in frame.lexUnit:
        context.update(lu.lower().split())
        # context.update(lu.lower().split('.')[0])
        context.update(frame.lexUnit[lu].definition.lower().split())
    # LUs = frame.lexUnit.keys()
    # lus = frame.lexUnit
    # for lu in LUs:
    #     context.update(lu.lower().split('.')[0])
    #     context.update(lus[lu].definition.lower().split())
    # for lu in frame.lexUnit:
    #     context.update(lu.name.lower().split())
    #     context.update(lu.definition.lower().split())
    #     for example in lu.example:
    #         context.update(example.lower().split())
    context = remove_stopwords(context)
    return context

context = get_frame_context(fn.frame('Eclipse'))
print(context)
print(len(context))
for elem in context:
    if elem in 'befogged.a':
        print(elem)


['befogged.a', 'conceal.v', 'subregion', 'observer,', 'shroud', 'occlude.v', 'element', 'body;', ".'", 'bungalow', 'close', 'indicate', 'degree', 'screen.v', 'total', 'momentarily', 'covered', 'mists', 'block;', 'wholly', 'opening', 'covered.a', 'veil.v', 'small', 'clouds', 'illumination', 'direct', 'visible', 'disguise,', 'screens', 'mask', 'shroud.v', 'alters', 'event', "foliage.'", 'flashed', 'green', 'shrouded', 'entity', 'obstruction', 'obscure', 'aesthetics', 'observers.', 'obscure.v', 'gate', 'eclipse.', 'house', 'hide', 'concealed', 'position', 'eclipse.v', 'occlusion.n', "tower.'", 'panel', 'obstruct', "'the", 'blocks', "side.'", 'shrouded.a', 'occultation.n', 'masked', 'obscured,', 'phrases', 'cloaked.a', 'veiled.a', 'cover', 'wearing', 'fence', 'view.', 'view', 'frame', 'big', 'masked.a', 'up,', 'source', 'conceal,', 'obscuring', 'top', 'allow', 'beclouded.a', 'partially', 'object', 'obstruction.', "road.'", 'blot', 'clouds.', "treetops.'", 'garden', 'shelter', 'veil', "euro

In [8]:
# Save the frameset in a dictionary
# frames = {
#     '2658': 'Suicide_attack',
#     '1633': 'Board_vehicle',
#     '1260': 'Simple_name',
#     '1871': 'Access_scenario',
#     '652': 'Eclipse'
# }

# for key in frames:
#     print(fn.frame(frames[key]))